In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/'My Drive'/UniversalEmb

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/UniversalEmb


In [2]:
!pip install transformers

     |████████████████████████████████| 645kB 8.1MB/s 
     |████████████████████████████████| 3.8MB 23.4MB/s 
     |████████████████████████████████| 1.0MB 61.1MB/s 
     |████████████████████████████████| 890kB 59.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=42504486d54c026c899996af2b10ee06c77b2d5242794d9eb945cfd27330736f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import pandas as pd
def read_trec50(file_name, typ=1):
  train = pd.read_csv(file_name, index_col=0)
  lbl2indx = {i:j for j, i in enumerate(train.labels.unique())}
  train.columns = ['text', 'labels']
  train['lbl_index'] = train.labels.apply(lambda x:lbl2indx[x])


  labels_t = [i.split(' ', 1)[0].split(':')[typ] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
  texts_t = [ i.split(' ', 1)[1][:-1] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
  test  = pd.DataFrame({'text':texts_t, 'labels': labels_t})
  test['lbl_index'] = test.labels.apply(lambda x:lbl2indx[x])

  return train, test, lbl2indx

train, test, l2i = read_trec50('trec50_splited/trec50_3k.csv')

In [0]:
import pandas as pd
def read_trec6(file_name, typ=0):
  train = pd.read_csv(file_name, index_col=0)
  lbl2indx = {i:j for j, i in enumerate(train.labels.unique())}
  train.columns = ['text', 'labels']
  train['lbl_index'] = train.labels.apply(lambda x:lbl2indx[x])


  labels_t = [i.split(' ', 1)[0].split(':')[typ] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
  texts_t = [ i.split(' ', 1)[1][:-1] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
  test  = pd.DataFrame({'text':texts_t, 'labels': labels_t})
  test['lbl_index'] = test.labels.apply(lambda x:lbl2indx[x])

  return train, test, lbl2indx
train, test, l2i = read_trec6('trec6_splited/trec6_splited_3k.csv')

In [0]:
import pandas as pd
train = pd.read_csv("sst2-splited/sst2_full_rev_3k.csv", index_col=0)
train.columns = ['text', 'lbl_index']

In [0]:
test = pd.read_csv("sst2-splited/test.csv", sep = '\t')
test.columns = ['text', 'lbl_index']

In [0]:
import torch
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [43]:
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
def prep_data(sentences, MAX_LEN = 64):
  input_ids = []
  for sent in tqdm(sentences):
      encoded_sent = tokenizer.encode(sent, add_special_tokens = True)
      input_ids.append(encoded_sent[:MAX_LEN])
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
  attention_masks = []
  for sent in input_ids:
      att_mask = [int(token_id > 0) for token_id in sent]
      attention_masks.append(att_mask)
  return input_ids, attention_masks

train_input_ids, train_attention_masks = prep_data(train.text.values.tolist())
test_input_ids, test_attention_masks = prep_data(test.text.values.tolist())

100%|██████████| 1821/1821 [00:00<00:00, 3347.99it/s]


In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_inputs = torch.tensor(train_input_ids)
validation_inputs = torch.tensor(test_input_ids)

train_labels = torch.tensor(train.lbl_index.values.tolist())
validation_labels = torch.tensor(test.lbl_index.values.tolist())

train_masks = torch.tensor(train_attention_masks)
validation_masks = torch.tensor(test_attention_masks)

batch_size = 16

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [0]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(list(train.lbl_index.unique())), # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = True # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model = model.cuda()

In [0]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 50, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))
device = 'cuda'

In [0]:
import keras.backend as K
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [51]:
import random
from sklearn.metrics import f1_score

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
loss_values = []

for epoch_i in range(0, epochs):
    embeddings = []

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()        

        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        loss  = outputs[0]
        embeddings.append(outputs[2][-1][:,0,:])
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    print("")
    print("Running Validation...")

    t0 = time.time()
    test_targets, test_pred_class = [], []

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        test_targets.append(label_ids)
        test_pred_class.append(np.argmax(logits, axis=1))
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        nb_eval_steps += 1
    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='weighted')

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  F1: {0:.4f}".format(f1))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training...
  Batch    40  of    188.    Elapsed: 0:00:08.
  Batch    80  of    188.    Elapsed: 0:00:17.
  Batch   120  of    188.    Elapsed: 0:00:26.
  Batch   160  of    188.    Elapsed: 0:00:35.

  Average training loss: 0.48
  Training epcoh took: 0:00:41

Running Validation...
  Accuracy: 0.89
  F1: 0.8851
  Validation took: 0:00:08

======== Epoch 2 / 3 ========
Training...
  Batch    40  of    188.    Elapsed: 0:00:08.
  Batch    80  of    188.    Elapsed: 0:00:17.
  Batch   120  of    188.    Elapsed: 0:00:25.
  Batch   160  of    188.    Elapsed: 0:00:34.

  Average training loss: 0.20
  Training epcoh took: 0:00:40

Running Validation...
  Accuracy: 0.89
  F1: 0.8901
  Validation took: 0:00:08

======== Epoch 3 / 3 ========
Training...
  Batch    40  of    188.    Elapsed: 0:00:09.
  Batch    80  of    188.    Elapsed: 0:00:17.
  Batch   120  of    188.    Elapsed: 0:00:26.
  Batch   160  of    188.    Elapsed: 0:00:34.

  Average training los

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

def getBertVectors(model, sentences, file_name, batch_size = 64, device = 'cuda'):
  input_ids, attention_masks = prep_data(sentences)

  inputs = torch.tensor(input_ids)
  labels = torch.tensor([0]*len(sentences))
  masks = torch.tensor(attention_masks)

  train_data = TensorDataset(inputs, masks, labels)
  train_sampler = RandomSampler(train_data)
  dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  embeddings = []
  model.eval()

  for batch in tqdm(dataloader):
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)
      model.zero_grad()        

      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
      embeddings.append(outputs[2][-1][:,0,:].detach().cpu().numpy())

  embeddings = np.concatenate(embeddings, axis = 0)
  np.save(file_name, embeddings)
  return embeddings
